In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Preprocessing

In [3]:
import pandas as pd

In [7]:
# Read the Redfin data into a DataFrame
file_path_redfin1 = "../data/redfin_data1.csv"
file_path_redfin2 = "../data/redfin_data2.csv"
df_redfin1 = pd.read_csv(file_path_redfin1)
df_redfin2 = pd.read_csv(file_path_redfin2)
df_redfin = pd.concat([df_redfin1, df_redfin2])
df_redfin["period_begin"] = pd.to_datetime(df_redfin["period_begin"])

#make it easier to merge with av_data
df_redfin["date"] = df_redfin["period_begin"] + pd.Timedelta(days = -3)
df_redfin = df_redfin.drop(columns=["period_begin", "period_end"])
df_redfin

,Unnamed: 0,region_name,adjusted_average_new_listings,adjusted_average_new_listings_yoy,average_pending_sales_listing_updates,average_pending_sales_listing_updates_yoy,off_market_in_two_weeks,off_market_in_two_weeks_yoy,adjusted_average_homes_sold,adjusted_average_homes_sold_yoy,...,months_of_supply,months_of_supply_yoy,median_pending_sqft,median_pending_sqft_yoy,average_sale_to_list_ratio,average_sale_to_list_ratio_yoy,median_sale_ppsf,median_sale_ppsf_yoy,State,date
0,2,Crook County,5.0,-0.375000,8.0,0.333333,1.0,-0.500000,9.0,3.500000,...,12.000000,-60.000000,1618.0,-0.071449,0.937917,0.021480,150.000000,0.218853,OR,2017-01-27
1,10,Chambers County,15.0,0.666667,14.0,0.076923,3.0,-0.250000,11.0,0.833333,...,13.363636,-10.469697,2227.0,0.183316,0.973733,-0.015372,94.778849,-0.035945,TX,2017-01-27
2,32,St. Joseph County,12.0,-0.555556,17.0,0.000000,1.0,-0.500000,13.0,-0.133333,...,20.153846,-3.512821,1500.0,-0.417363,0.933705,0.005458,61.458333,0.156474,MI,2017-02-03
3,42,Crook County,6.0,-0.400000,12.0,-0.142857,1.0,-0.500000,1.0,-0.888889,...,107.000000,90.666667,1984.0,0.634267,1.000000,-0.009948,143.509615,0.506180,OR,2017-02-03
4,69,Coos County,16.0,-0.360000,23.0,-0.080000,5.0,-0.166667,11.0,-0.450000,...,28.818182,13.268182,1760.0,0.085415,0.941134,-0.017685,116.892373,-0.041654,OR,2017-03-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166274,4868262,Troup County,15.0,0.363636,20.0,1.857143,1.0,0.000000,18.0,0.200000,...,14.222222,-3.111111,1648.0,-0.208833,0.988753,-0.010559,146.915071,0.086005,GA,2024-11-15
166275,4868265,Nez Perce County,7.0,0.400000,6.0,0.200000,2.0,-0.333333,8.0,1.000000,...,12.375000,-10.875000,2860.0,0.914324,0.999153,0.041135,189.274448,-0.264456,ID,2024-11-15
166276,4868271,Coles County,17.0,1.833333,11.0,0.222222,4.0,3.000000,9.0,2.000000,...,10.555556,-19.777778,1519.5,-0.038291,0.941275,-0.056837,104.166667,0.351781,IL,2024-11-15
166277,4868277,Marion County,6.0,-0.142857,4.0,-0.500000,4.0,1.000000,12.0,1.400000,...,8.166667,-6.433333,1098.5,-0.147127,0.966881,-0.046709,214.650323,-0.145966,IA,2024-11-15


In [8]:
#Inspect Data
df_redfin.info()

<class 'pandas.core.frame.DataFrame'>
Index: 332558 entries, 0 to 166278
Data columns (total 36 columns):
 #   Column                                        Non-Null Count   Dtype         
---  ------                                        --------------   -----         
 0   Unnamed: 0                                    332558 non-null  int64         
 1   region_name                                   332558 non-null  object        
 2   adjusted_average_new_listings                 332558 non-null  float64       
 3   adjusted_average_new_listings_yoy             332558 non-null  float64       
 4   average_pending_sales_listing_updates         332558 non-null  float64       
 5   average_pending_sales_listing_updates_yoy     332558 non-null  float64       
 6   off_market_in_two_weeks                       332558 non-null  float64       
 7   off_market_in_two_weeks_yoy                   332558 non-null  float64       
 8   adjusted_average_homes_sold                   332558 non-nu

In [9]:
# Read the Zillow data into a DataFrame
file_path_zillow_zori = "../data/County_zori_uc_sfrcondomfr_sm_month.csv"
file_path_zillow_zhvi = "../data/County_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv"
df_zillow_zori = pd.read_csv(file_path_zillow_zori)
df_zillow_zhvi = pd.read_csv(file_path_zillow_zhvi)

In [10]:
#Extract required features
df_zillow_zori.drop(columns=["RegionID", "SizeRank", "StateName", "RegionType", "StateCodeFIPS", "MunicipalCodeFIPS"], inplace=True)
df_zillow_zhvi.drop(columns=["RegionID", "SizeRank", "StateName", "RegionType", "StateCodeFIPS", "MunicipalCodeFIPS"], inplace=True)

#restructure so that the dates become rows
df_zillow_zori = pd.melt(df_zillow_zori, id_vars=["RegionName", "State", "Metro"],
                        var_name="Date",
                        value_name="ZORI")
df_zillow_zhvi = pd.melt(df_zillow_zhvi, id_vars=["RegionName", "State", "Metro"],
                        var_name="Date",
                        value_name="ZHVI")

#apply time horizon
df_zillow_zori["Date"] = pd.to_datetime(df_zillow_zori["Date"], infer_datetime_format = True)
df_zillow_zhvi["Date"] = pd.to_datetime(df_zillow_zhvi["Date"], infer_datetime_format = True)
df_zillow_zhvi = df_zillow_zhvi[df_zillow_zhvi["Date"] > "2015-01-31"]

#merge
df_zillow = pd.merge(df_zillow_zhvi, df_zillow_zori[["RegionName", "State", "Metro", "Date", "ZORI"]],
                     on = ["RegionName", "State", "Metro", "Date"],
                     how = "left")
df_zillow.dropna(inplace=True)

#rename column to match df_redfin
df_zillow.rename(columns={"RegionName": "region_name"}, inplace=True)
df_zillow.rename(columns={"Date": "date"}, inplace=True)

#resample to weekly
df_zillow["date"] += pd.to_timedelta(df_zillow.groupby("date").cumcount(), unit = "s")
df_zillow.set_index("date", inplace=True)
df_zillow = df_zillow.groupby("region_name").resample("W-FRI").ffill()
df_zillow = df_zillow.drop(columns=["region_name"]).reset_index()
df_zillow

<ipython-input-10-a391ab3e0e27>:14: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_zillow_zori["Date"] = pd.to_datetime(df_zillow_zori["Date"], infer_datetime_format = True)
<ipython-input-10-a391ab3e0e27>:15: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_zillow_zhvi["Date"] = pd.to_datetime(df_zillow_zhvi["Date"], infer_datetime_format = True)


,region_name,date,State,Metro,ZHVI,ZORI
0,Ada County,2015-03-06,ID,"Boise City, ID",204657.323819,948.248180
1,Ada County,2015-03-13,ID,"Boise City, ID",204657.323819,948.248180
2,Ada County,2015-03-20,ID,"Boise City, ID",204657.323819,948.248180
3,Ada County,2015-03-27,ID,"Boise City, ID",204657.323819,948.248180
4,Ada County,2015-04-03,ID,"Boise City, ID",205924.714086,956.009824
...,...,...,...,...,...,...
250238,Yuma County,2024-10-04,AZ,"Yuma, AZ",277673.650150,1473.600844
250239,Yuma County,2024-10-11,AZ,"Yuma, AZ",277673.650150,1473.600844
250240,Yuma County,2024-10-18,AZ,"Yuma, AZ",277673.650150,1473.600844
250241,Yuma County,2024-10-25,AZ,"Yuma, AZ",277673.650150,1473.600844


In [11]:
#Inspect Data
df_zillow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250243 entries, 0 to 250242
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   region_name  250243 non-null  object        
 1   date         250243 non-null  datetime64[ns]
 2   State        250198 non-null  object        
 3   Metro        250198 non-null  object        
 4   ZHVI         250198 non-null  float64       
 5   ZORI         250198 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 11.5+ MB


In [5]:
#Read alpha vantage data
file_path_av = "../data/av_data.csv"
df_av = pd.read_csv(file_path_av)
df_av["date"] = pd.to_datetime(df_av["date"])
df_av

,date,SPY_adj_close,SPY_return_1day,SPY_return_1week,SPY_return_1month,SPY_return_3month,EQR_adj_close,EQR_adj_close_1day,EQR_return_1day,ESS_adj_close,...,AVB_adj_close,AVB_adj_close_1day,AVB_return_1day,INVH_adj_close,INVH_adj_close_1day,INVH_return_1day,unemployment,fed_funds,3monhth_treasury_yield,cpi
0,2017-02-03,201.410539,-0.001788,0.009462,0.036496,0.045656,45.852975,45.989668,0.002981,173.412006,...,133.776941,134.771422,0.007434,17.272979,17.356707,0.004847,0.047,0.000098,0.0065,2.42839
1,2017-02-10,203.316271,0.005443,0.015464,0.027213,0.038761,47.030055,47.189531,0.003391,177.531979,...,138.664545,137.646936,-0.007339,17.599516,17.582771,-0.000951,0.046,0.000104,0.0066,2.43603
2,2017-02-17,206.460293,0.005955,0.008592,0.011536,0.025677,46.764263,47.402165,0.013641,177.314320,...,137.639227,139.612770,0.014339,17.708362,17.976290,0.015130,0.046,0.000102,0.0066,2.43603
3,2017-02-24,207.909361,0.001563,0.007096,-0.007860,0.016153,47.675551,48.040066,0.007646,180.618072,...,141.123765,141.455258,0.002349,18.085136,18.260964,0.009722,0.046,0.000100,0.0066,2.43603
4,2017-03-03,209.384761,-0.002978,-0.003062,-0.006931,0.017333,47.326224,47.402165,0.001605,179.218836,...,139.666735,140.946454,0.009163,18.260964,18.353064,0.005044,0.046,0.000117,0.0065,2.43603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381,2024-08-16,552.612504,0.009561,0.014108,0.015389,0.083508,70.775391,70.953791,0.002521,281.673999,...,213.597940,214.421723,0.003857,35.000079,35.347302,0.009921,0.042,0.000999,0.0533,3.14796
382,2024-08-23,560.408556,-0.002384,0.002757,0.016526,0.049502,72.975662,72.281883,-0.009507,293.654641,...,220.069101,218.272659,-0.008163,35.863177,35.545716,-0.008852,0.042,0.000990,0.0533,3.14796
383,2024-08-30,561.953809,-0.020579,-0.030638,0.021009,0.063308,74.214553,74.987622,0.010417,299.307758,...,224.039139,225.438577,0.006246,36.547702,36.974290,0.011672,0.042,0.000979,0.0533,3.14796
384,2024-09-06,538.705224,0.011196,0.040066,0.063624,0.120520,73.679352,74.432598,0.010223,295.757203,...,221.925094,223.800936,0.008453,35.664763,35.406826,-0.007232,0.041,0.000968,0.0533,3.15301


In [6]:
#Inspect Data
df_av.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386 entries, 0 to 385
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   date                    386 non-null    datetime64[ns]
 1   SPY_adj_close           386 non-null    float64       
 2   SPY_return_1day         386 non-null    float64       
 3   SPY_return_1week        386 non-null    float64       
 4   SPY_return_1month       386 non-null    float64       
 5   SPY_return_3month       386 non-null    float64       
 6   EQR_adj_close           386 non-null    float64       
 7   EQR_adj_close_1day      386 non-null    float64       
 8   EQR_return_1day         386 non-null    float64       
 9   ESS_adj_close           386 non-null    float64       
 10  ESS_adj_close_1day      386 non-null    float64       
 11  ESS_return_1day         386 non-null    float64       
 12  AVB_adj_close           386 non-null    float64   

In [14]:
#merge three data source
df = pd.merge(df_redfin, df_zillow[["region_name", "State", "date" , "ZORI", "ZHVI"]],
                     on = ["region_name", "State", "date"],
                     how = "left")

df = pd.merge(df, df_av,
                     on = [ "date"],
                     how = "left")
df.dropna(inplace=True)
df = df.drop(columns=["Unnamed: 0"])
df

,region_name,adjusted_average_new_listings,adjusted_average_new_listings_yoy,average_pending_sales_listing_updates,average_pending_sales_listing_updates_yoy,off_market_in_two_weeks,off_market_in_two_weeks_yoy,adjusted_average_homes_sold,adjusted_average_homes_sold_yoy,median_new_listing_price,...,AVB_adj_close,AVB_adj_close_1day,AVB_return_1day,INVH_adj_close,INVH_adj_close_1day,INVH_return_1day,unemployment,fed_funds,3monhth_treasury_yield,cpi
16,Stanislaus County,163.0,0.108844,152.0,-0.061728,52.0,-0.187500,112.0,-0.157895,289000.0,...,137.045622,137.230642,0.001350,17.917681,17.900935,-0.000935,0.044,0.000142,0.0066,2.43801
18,Stearns County,73.0,0.123077,42.0,0.000000,11.0,0.100000,31.0,0.033333,168500.0,...,137.045622,137.230642,0.001350,17.917681,17.900935,-0.000935,0.044,0.000142,0.0066,2.43801
38,Denver County,335.0,-0.079670,313.0,-0.163102,162.0,0.045161,341.0,0.249084,399900.0,...,150.652722,150.535330,-0.000779,17.908001,17.899605,-0.000469,0.043,0.000216,0.0116,2.44786
45,Stanislaus County,120.0,-0.097744,165.0,0.071429,44.0,-0.185185,76.0,-0.296296,279975.0,...,137.639227,139.612770,0.014339,17.708362,17.976290,0.015130,0.046,0.000102,0.0066,2.43603
47,Stearns County,50.0,-0.107143,46.0,0.210526,6.0,0.500000,16.0,-0.238095,237450.0,...,137.639227,139.612770,0.014339,17.708362,17.976290,0.015130,0.046,0.000102,0.0066,2.43603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332534,Cuyahoga County,397.0,-0.076744,405.0,-0.093960,150.0,-0.202128,292.0,-0.165714,202950.0,...,198.744645,198.459163,-0.001436,34.941491,34.951334,0.000282,0.041,0.001032,0.0533,3.14175
332536,Butler County,14.0,-0.300000,15.0,-0.062500,6.0,0.200000,20.0,0.250000,243400.0,...,208.883521,207.791760,-0.005227,35.406826,34.970317,-0.012328,0.042,0.001001,0.0533,3.14796
332539,Schuylkill County,40.0,0.333333,34.0,0.096774,12.0,-0.076923,30.0,0.200000,124950.0,...,198.744645,198.459163,-0.001436,34.941491,34.951334,0.000282,0.041,0.001032,0.0533,3.14175
332540,Aransas County,14.0,0.400000,14.0,0.555556,0.0,-1.000000,16.0,1.000000,389500.0,...,198.744645,198.459163,-0.001436,34.941491,34.951334,0.000282,0.041,0.001032,0.0533,3.14175


In [15]:
df.describe()

,adjusted_average_new_listings,adjusted_average_new_listings_yoy,average_pending_sales_listing_updates,average_pending_sales_listing_updates_yoy,off_market_in_two_weeks,off_market_in_two_weeks_yoy,adjusted_average_homes_sold,adjusted_average_homes_sold_yoy,median_new_listing_price,median_new_listing_price_yoy,...,AVB_adj_close,AVB_adj_close_1day,AVB_return_1day,INVH_adj_close,INVH_adj_close_1day,INVH_return_1day,unemployment,fed_funds,3monhth_treasury_yield,cpi
count,125694.000000,125694.000000,125694.000000,125694.000000,125694.000000,125694.000000,125694.000000,125694.000000,1.256940e+05,125694.000000,...,125694.000000,125694.000000,125694.000000,125694.000000,125694.000000,125694.000000,125694.000000,125694.000000,125694.000000,125694.000000
mean,145.053423,0.021724,147.495409,0.005742,39.808400,0.235575,123.976546,0.005638,3.738199e+05,0.080283,...,170.905422,170.722698,-0.000822,28.691635,28.661343,-0.000817,0.044231,0.000514,0.026030,2.802545
min,1.000000,-0.956349,1.000000,-0.995088,0.000000,-1.000000,1.000000,-0.941176,2.657100e+04,-0.930806,...,111.523051,102.592395,-0.153707,15.199123,14.261888,-0.235490,0.034000,0.000002,0.000400,2.428390
25%,38.000000,-0.172414,40.000000,-0.183333,7.000000,-0.333333,33.000000,-0.200000,2.400000e+05,-0.002719,...,149.596196,149.467066,-0.008436,23.398654,23.395988,-0.007232,0.036000,0.000148,0.006600,2.563940
50%,79.000000,-0.018182,81.000000,-0.037522,18.000000,0.000000,69.000000,-0.038462,3.224500e+05,0.067276,...,171.066776,170.304496,-0.000249,30.442942,30.405029,-0.000291,0.038000,0.000419,0.021800,2.779480
75%,166.000000,0.147059,171.000000,0.122449,45.000000,0.400000,145.000000,0.138298,4.374500e+05,0.150000,...,186.863938,186.890886,0.007351,33.901276,34.008014,0.006421,0.043000,0.001006,0.050800,3.056910
max,2608.000000,22.818182,2649.000000,154.000000,1165.000000,57.000000,2679.000000,16.300000,1.075000e+07,8.673529,...,232.991573,231.115730,0.113146,41.540024,41.201033,0.107383,0.148000,0.001052,0.053300,3.153010
std,216.049788,0.365648,217.480390,0.669485,65.274167,1.359347,180.224485,0.360610,2.243091e+05,0.162633,...,25.939227,25.796172,0.017719,6.463951,6.448792,0.019194,0.017383,0.000401,0.020722,0.250521


In [17]:
#move date to the first column
df.sort_values(by=["date"], inplace=True)
column_to_move = "date"
cols = [column_to_move] + [col for col in df.columns if col != column_to_move]
df = df[cols]
df

,date,region_name,adjusted_average_new_listings,adjusted_average_new_listings_yoy,average_pending_sales_listing_updates,average_pending_sales_listing_updates_yoy,off_market_in_two_weeks,off_market_in_two_weeks_yoy,adjusted_average_homes_sold,adjusted_average_homes_sold_yoy,...,AVB_adj_close,AVB_adj_close_1day,AVB_return_1day,INVH_adj_close,INVH_adj_close_1day,INVH_return_1day,unemployment,fed_funds,3monhth_treasury_yield,cpi
13371,2017-02-03,Santa Barbara County,62.0,-0.318681,55.0,-0.017857,11.0,0.375000,32.0,-0.428571,...,133.776941,134.771422,0.007434,17.272979,17.356707,0.004847,0.047,0.000098,0.0065,2.42839
1802,2017-02-03,Hendricks County,46.0,-0.021277,71.0,-0.065789,13.0,3.333333,33.0,0.320000,...,133.776941,134.771422,0.007434,17.272979,17.356707,0.004847,0.047,0.000098,0.0065,2.42839
1788,2017-02-03,Collier County,394.0,-0.048309,336.0,2.000000,60.0,0.395349,128.0,0.122807,...,133.776941,134.771422,0.007434,17.272979,17.356707,0.004847,0.047,0.000098,0.0065,2.42839
8862,2017-02-03,Stafford County,74.0,0.174603,41.0,-0.108696,21.0,0.105263,22.0,-0.214286,...,133.776941,134.771422,0.007434,17.272979,17.356707,0.004847,0.047,0.000098,0.0065,2.42839
8861,2017-02-03,Macomb County,242.0,-0.312500,305.0,0.089286,103.0,0.303797,188.0,-0.005291,...,133.776941,134.771422,0.007434,17.272979,17.356707,0.004847,0.047,0.000098,0.0065,2.42839
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284915,2024-09-13,Anne Arundel County,167.0,0.159722,173.0,0.176871,63.0,0.465116,150.0,0.020408,...,232.991573,231.115730,-0.008051,36.726274,36.478257,-0.006753,0.041,0.000950,0.0533,3.15301
284910,2024-09-13,Walton County,94.0,0.080460,56.0,-0.034483,3.0,0.500000,57.0,-0.109375,...,232.991573,231.115730,-0.008051,36.726274,36.478257,-0.006753,0.041,0.000950,0.0533,3.15301
284907,2024-09-13,San Bernardino County,429.0,0.004684,379.0,-0.152125,63.0,-0.073529,374.0,0.093567,...,232.991573,231.115730,-0.008051,36.726274,36.478257,-0.006753,0.041,0.000950,0.0533,3.15301
285105,2024-09-13,Contra Costa County,284.0,0.296804,232.0,-0.012766,90.0,-0.203540,188.0,-0.055276,...,232.991573,231.115730,-0.008051,36.726274,36.478257,-0.006753,0.041,0.000950,0.0533,3.15301
